# Part 3 Final Exam

Mash up data from two different APIs to make movie recommendations.
* The TasteDive API lets you provide a movie (or bands, TV shows, etc.) as a query input, and returns a set of related items.
* The OMDB API lets you provide a movie title as a query input and get back data about the movie, including scores from various review sites (Rotten Tomatoes, IMDB, etc.).

You will put those two together.
* You will use TasteDive to get related movies for a whole list of titles.
* You’ll combine the resulting lists of related movies, and sort them according to their Rotten Tomatoes scores by making API call to the OMDB API.

## Step 1

### TasteDive
* https://tastedive.com/read/api

* Define a function, called `get_movies_from_tastedive`.
* It should take one input parameter, a string that is the name of a movie or music artist.
* The function should return the 5 TasteDive results that are associated with that string
* Be sure to only get movies, not other kinds of media.
* It will be a python dictionary with just one key, `Similar`.

Try invoking your function with the input `“Black Panther”`.

HINT: Be sure to include only `q`, `type`, and `limit` as parameters in order to extract data from the cache. If any other parameters are included, then the function will not be able to recognize the data that you’re attempting to pull from the cache. Remember, you will not need an api key in order to complete the project, because all data will be found in the cache.



In [29]:
import requests
import json

td_api_key = '' # Taste Dive key quota is 300

# retrieve API key from file
with open('tastedive.json', 'r') as f:
    td_api_key = json.load(f)['key']
    
def get_movies_from_tastedive(q):
    #q is a string containing music artist, movie, author, book, etc. (i.e. 'Red Hot Chili Peppers')
    #q can have an identifier if, say, book and movie have same name (i.e. 'book:Harry Potter')
    td_endpoint = 'https://tastedive.com/api/similar'
    params = {
        'k' : td_api_key,
        'q' : q,
        'type' : 'movies', #(optional) music, movies, shows, podcasts, books, authors, games
        'limit' : '5' #(optional) maximum number of recommendations to retrieve (default is 20)
    }
    res = requests.get(td_endpoint, params=params)
    print(res)
    print(res.url)
    return res.json()

td_results = get_movies_from_tastedive('Black Panther')
print(json.dumps(td_results, indent = 2))


<Response [200]>
https://tastedive.com/api/similar?k=444192-practice-DRUD67MN&q=Black+Panther&type=movies&limit=5
{
  "Similar": {
    "Info": [
      {
        "Name": "Black Panther",
        "Type": "movie"
      }
    ],
    "Results": [
      {
        "Name": "Avengers: Infinity War",
        "Type": "movie"
      },
      {
        "Name": "Ant-Man And The Wasp",
        "Type": "movie"
      },
      {
        "Name": "Thor: Ragnarok",
        "Type": "movie"
      },
      {
        "Name": "Guardians Of The Galaxy Vol. 2",
        "Type": "movie"
      },
      {
        "Name": "Captain Marvel",
        "Type": "movie"
      }
    ]
  }
}


Next, you will need to write a function that extracts just the list of movie titles from a dictionary returned by `get_movies_from_tastedive`. Call it `extract_movie_titles`.

In [37]:
def extract_movie_titles(td_d):
    #td_d is the dictionary returned by get_movies_from_tastedive()
    movies = td_d["Similar"]["Results"]
    return [m['Name'] for m in movies]

In [38]:
print(extract_movie_titles(td_results))

['Avengers: Infinity War', 'Ant-Man And The Wasp', 'Thor: Ragnarok', 'Guardians Of The Galaxy Vol. 2', 'Captain Marvel']


Next, you’ll write a function, called `get_related_titles`. It takes a list of movie titles as input. It gets five related movies for each from TasteDive, extracts the titles for all of them, and combines them all into a single list. Don’t include the same movie twice.

In [65]:
def get_related_titles(movie_titles_l):
    #movie_titles_l is a list of movie titles that you will get related movies from
    all_results = []
    for t in movie_titles_l:
        td_res = get_movies_from_tastedive(t)
        rel_movies = extract_movie_titles(td_res)
        [all_results.append(m) for m in rel_movies if m not in all_results]
    return all_results

print(get_related_titles(['Black Panther', 'Captain Marvel']))

<Response [200]>
https://tastedive.com/api/similar?k=444192-practice-DRUD67MN&q=Black+Panther&type=movies&limit=5
<Response [200]>
https://tastedive.com/api/similar?k=444192-practice-DRUD67MN&q=Captain+Marvel&type=movies&limit=5
['Avengers: Infinity War', 'Ant-Man And The Wasp', 'Thor: Ragnarok', 'Guardians Of The Galaxy Vol. 2', 'Captain Marvel', 'Spider-Man: Far From Home', 'Avengers: Endgame']


## Step 2:

### OMDb

Your next task will be to fetch data from __OMDB__. The documentation for the API is at https://www.omdbapi.com/

Define a function called `get_movie_data`. It takes in __one parameter__ which is a string that should represent the __title of a movie__ you want to search. The function should __return a dictionary with information about that movie__.

You will need to provide the following parameters: `t` and `r`.

In [71]:
# get OMDb key from JSON file
omdb_key = ''
with open('OMDb.json', 'r') as f:
    omdb_key = json.load(f)['key']
print(omdb_key)

4ef2cb3b


In [81]:
def get_movie_data(t):
    #t is tile of movie to search
    endpoint = 'http://www.omdbapi.com/'
    params = {
        'apikey' : '4ef2cb3b',
        't' : t, #title of movie to search for
        'r' : 'json' #data type to return
    }
    res = requests.get(endpoint, params=params)
    #http://www.omdbapi.com/?t=Black+Panther&apikey=4ef2cb3b
    print(res.url)
    return res.json()

In [85]:
o = get_movie_data('Black Panther')
print(json.dumps(o, indent=2))

http://www.omdbapi.com/?apikey=4ef2cb3b&t=Black+Panther&r=json
{
  "Title": "Black Panther",
  "Year": "2018",
  "Rated": "PG-13",
  "Released": "16 Feb 2018",
  "Runtime": "134 min",
  "Genre": "Action, Adventure, Sci-Fi",
  "Director": "Ryan Coogler",
  "Writer": "Ryan Coogler, Joe Robert Cole, Stan Lee",
  "Actors": "Chadwick Boseman, Michael B. Jordan, Lupita Nyong'o",
  "Plot": "T'Challa, heir to the hidden but advanced kingdom of Wakanda, must step forward to lead his people into a new future and must confront a challenger from his country's past.",
  "Language": "English, Swahili, Nama, Xhosa, Korean",
  "Country": "United States",
  "Awards": "Won 3 Oscars. 116 wins & 281 nominations total",
  "Poster": "https://m.media-amazon.com/images/M/MV5BMTg1MTY2MjYzNV5BMl5BanBnXkFtZTgwMTc4NTMwNDI@._V1_SX300.jpg",
  "Ratings": [
    {
      "Source": "Internet Movie Database",
      "Value": "7.3/10"
    },
    {
      "Source": "Rotten Tomatoes",
      "Value": "96%"
    },
    {
      "

Now write a function called `get_movie_rating`. It takes an OMDB dictionary result for one movie and extracts the Rotten Tomatoes rating as an integer. For example, if given the OMDB dictionary for “Black Panther”, it would return 97. If there is no Rotten Tomatoes rating, return 0.

In [100]:
def get_movie_rating(omdb_result_d):
    rating = 0
    rating_l = omdb_result_d['Ratings'] #returns a list of objects ("Source":"Value")
    for r in rating_l:
        if (r['Source'] == 'Rotten Tomatoes'):
            rating = int(r['Value'][:-1])
            break
    return rating


In [105]:
get_movie_rating(o)

96

## Step 3
### Merge all results into a sorted list

Now, you’ll put it all together.

Define a function `get_sorted_recommendations`. It takes a __list of movie titles__ as an input. It __returns__ a __sorted list__ of __related movie titles__ as output, up to five related movies for each input movie title.

The movies should be sorted in __descending order__ by their __Rotten Tomatoes rating__, as returned by the `get_movie_rating` function. __Break ties in reverse alphabetic order__, so that ‘Yahşi Batı’ comes before ‘Eyyvah Eyvah’.

In [142]:
def get_sorted_recommendations(movie_titles_l):
    #movie_titles_l is a list of movie titles to get related movie titles on
    rel_titles_l = get_related_titles(movie_titles_l) # get a filtered list of related movie titles
    rating_l = [] # collects list of ratings for the movie titles
    for title in rel_titles_l:
        data_d = get_movie_data(title)
        rating_l.append(get_movie_rating(data_d))
    collection = list(zip(rel_titles_l, rating_l))
    sort_collection = sorted(collection, reverse=True, key=lambda c : (c[1], c[0]))
    sorted_movies_t = list(zip(*sort_collection))[0]
    return [s for s in sorted_movies_t]
    
get_sorted_recommendations(['Black Panther', 'Captain America'])

<Response [200]>
https://tastedive.com/api/similar?k=444192-practice-DRUD67MN&q=Black+Panther&type=movies&limit=5
<Response [200]>
https://tastedive.com/api/similar?k=444192-practice-DRUD67MN&q=Captain+America&type=movies&limit=5
http://www.omdbapi.com/?apikey=4ef2cb3b&t=Avengers%3A+Infinity+War&r=json
http://www.omdbapi.com/?apikey=4ef2cb3b&t=Ant-Man+And+The+Wasp&r=json
http://www.omdbapi.com/?apikey=4ef2cb3b&t=Thor%3A+Ragnarok&r=json
http://www.omdbapi.com/?apikey=4ef2cb3b&t=Guardians+Of+The+Galaxy+Vol.+2&r=json
http://www.omdbapi.com/?apikey=4ef2cb3b&t=Captain+Marvel&r=json
http://www.omdbapi.com/?apikey=4ef2cb3b&t=The+Fantastic+Four&r=json
http://www.omdbapi.com/?apikey=4ef2cb3b&t=Batman+%26+Robin&r=json
http://www.omdbapi.com/?apikey=4ef2cb3b&t=Highlander+II%3A+The+Quickening&r=json
http://www.omdbapi.com/?apikey=4ef2cb3b&t=Double+Dragon&r=json
http://www.omdbapi.com/?apikey=4ef2cb3b&t=Highlander%3A+The+Source&r=json


['Thor: Ragnarok',
 'Ant-Man And The Wasp',
 'Guardians Of The Galaxy Vol. 2',
 'Avengers: Infinity War',
 'Captain Marvel',
 'The Fantastic Four',
 'Double Dragon',
 'Batman & Robin',
 'Highlander: The Source',
 'Highlander II: The Quickening']

In [114]:
rel_titles_l = ['Avengers: Infinity War', 'Ant-Man And The Wasp', 'Thor: Ragnarok', 'Guardians Of The Galaxy Vol. 2', 'Captain Marvel', 'The Fantastic Four', 'Batman & Robin', 'Highlander II: The Quickening', 'Double Dragon', 'Highlander: The Source']
rating_l = [85, 87, 93, 85, 79, 27, 12, 0, 12, 0]


In [119]:
z = list(zip(rel_titles_l,rating_l))
z

[('Avengers: Infinity War', 85),
 ('Ant-Man And The Wasp', 87),
 ('Thor: Ragnarok', 93),
 ('Guardians Of The Galaxy Vol. 2', 85),
 ('Captain Marvel', 79),
 ('The Fantastic Four', 27),
 ('Batman & Robin', 12),
 ('Highlander II: The Quickening', 0),
 ('Double Dragon', 12),
 ('Highlander: The Source', 0)]

In [134]:
z_sort = sorted(z, key=lambda x : (x[1], x[0]), reverse=True)
z_sort

[('Thor: Ragnarok', 93),
 ('Ant-Man And The Wasp', 87),
 ('Guardians Of The Galaxy Vol. 2', 85),
 ('Avengers: Infinity War', 85),
 ('Captain Marvel', 79),
 ('The Fantastic Four', 27),
 ('Double Dragon', 12),
 ('Batman & Robin', 12),
 ('Highlander: The Source', 0),
 ('Highlander II: The Quickening', 0)]